In [1]:
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
from tqdm import tqdm
from fuzzywuzzy import fuzz
import os

In [2]:
data_format  = 'morph'

train_df = pd.read_csv(f'originial_unclean_data/{data_format}/train.tsv', sep='\t')
test_df = pd.read_csv(f'originial_unclean_data/{data_format}/test_gold.tsv', sep='\t')
dev_df = pd.read_csv(f'originial_unclean_data/{data_format}/dev.tsv', sep='\t')


In [3]:
train_df = train_df.reset_index()
test_df = test_df.reset_index()
dev_df = dev_df.reset_index() 

In [4]:
train_df['type'] = 'train'
test_df['type'] = 'test'
dev_df['type'] = 'dev'

In [5]:
assert len(dev_df[dev_df['sentence'].str.contains("\t")]) == 0

In [6]:
train_df.drop_duplicates(subset=['sentence'])

,index,sentence,label,type
0,0,ממש כואב ..... אני בכה את היא עם ה משפחה ש לא ...,0,train
1,1,איש יקר של אנחנו,0,train
2,2,כל ה כבוד ו המון ב הצלחה,0,train
3,3,"תל חי , רובי . ב כל ה עצב ה זה היית קרן אור ש...",0,train
4,4,נקי כפיים ו בר לבב ב הצלחה ל אתה ו ל אנחנו .,0,train
...,...,...,...,...
9215,9215,ל מדינת ישראל יש נשיא סוף סוף,0,train
9216,9216,"יישר כח , אדוני ה נשיא < 3",0,train
9217,9217,"איזה כיף , ב הצלחה איש יקר",0,train
9218,9218,אין מועמד ראוי מ מך.בהצלחה !!,0,train


In [7]:
dfs = [train_df, test_df, dev_df]
dedup_df = []

In [8]:
for name, d in zip(["train_sen", "dev_sen", "test_sen"], dfs):
    d_dedup = d.drop_duplicates(subset=['sentence'])
    print(f"Before dedup {name}:", len(d))
    print(f"After dedup {name}:", len(d_dedup))
    print(f"Dedupd {name}:", len(d) - len(d_dedup))
    print(f"Precentage: {name}: {((len(d) - len(d_dedup)) / len(d)) * 100:.2f}%")
    dedup_df.append(d_dedup)

Before dedup train_sen: 9221
After dedup train_sen: 6953
Dedupd train_sen: 2268
Precentage: train_sen: 24.60%
Before dedup dev_sen: 2560
After dedup dev_sen: 2367
Dedupd dev_sen: 193
Precentage: dev_sen: 7.54%
Before dedup test_sen: 1023
After dedup test_sen: 984
Dedupd test_sen: 39
Precentage: test_sen: 3.81%


In [9]:
all_dedup = pd.concat(dedup_df)
final = all_dedup.drop_duplicates(subset=['sentence'])

In [10]:
final

,index,sentence,label,type
0,0,ממש כואב ..... אני בכה את היא עם ה משפחה ש לא ...,0,train
1,1,איש יקר של אנחנו,0,train
2,2,כל ה כבוד ו המון ב הצלחה,0,train
3,3,"תל חי , רובי . ב כל ה עצב ה זה היית קרן אור ש...",0,train
4,4,נקי כפיים ו בר לבב ב הצלחה ל אתה ו ל אנחנו .,0,train
...,...,...,...,...
1004,1004,ב הצלחה רבה מגיע ל הוא,0,dev
1012,1012,"מ ""ס 1 מגיע ל אתה להיות ה נשיא של אנחנו",0,dev
1013,1013,שבת שלום ל נשיא עם הכי הרבה קונצנזוס : ) ב הצל...,0,dev
1015,1015,גם ל אתם שבת שלום ו מבורכת ל אתם,0,dev


In [11]:
final['sentence'] = final['sentence'].apply(lambda x: x.strip("\""))

<ipython-input-11-2f2a6f3b8bae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['sentence'] = final['sentence'].apply(lambda x: x.strip("\""))


In [12]:
to_dedup = list(zip(final['sentence'], final['label'], final['index'],final['type'] ))

In [13]:
unique_comment = set()
similarity_group = {}
groups = {}
last_group = 0
for i, t1 in tqdm(enumerate(to_dedup)):
    unique = True
    for j, t2 in enumerate(to_dedup):
        if abs(len(t1[0]) - len(t2[0])) > 5 or  t2 in unique_comment or t1==t2 or t2 in groups:
            continue
        if len(t1[0]) < 7 and len(t2[0]) < 7:
            is_similar =  t1[0].strip('!,.? ') == t2[0].strip('!,.? ')
        else:
            try:
                is_similar = fuzz.partial_ratio(t1[0], t2[0]) > 95
            except ValueError:
                is_similar = SequenceMatcher(None, t1[0], t2[0]).ratio() > 0.95
        if is_similar:
            unique = False
            if t1 in groups:
                similarity_group[groups[t1]].append(t2)
                groups[t2] = groups[t1]
            else:
                groups[t1] = last_group
                groups[t2] = last_group
                similarity_group[last_group] = [t1, t2]
                last_group +=1
    if unique:
        unique_comment.add(t1)

8623it [36:14,  3.96it/s]


In [14]:
result = [{'comment': comment, 'label': label, 'index': index, 'type': set_type} for comment, label, index, set_type in unique_comment]

In [15]:
res_dedup = pd.DataFrame(result)

In [16]:
def class2int(c):
    d = {'dev': 1, 'train':0, 'test': 2}
    return d[c]
res_dedup['type'] = res_dedup['type'].apply(class2int) #
res_dedup.sort_values(['type', 'index' ])

,comment,label,index,type
2602,ממש כואב ..... אני בכה את היא עם ה משפחה ש לא ...,0,0,0
4787,איש יקר של אנחנו,0,1,0
3095,כל ה כבוד ו המון ב הצלחה,0,2,0
3093,"תל חי , רובי . ב כל ה עצב ה זה היית קרן אור ש...",0,3,0
2579,נקי כפיים ו בר לבב ב הצלחה ל אתה ו ל אנחנו .,0,4,0
...,...,...,...,...
7409,נשיא מדינת ישראל-תואר ש רובי ריבלין ראוי ל הוא...,0,2551,2
2024,ב סד נשמח ב ה שבוע . ה בא על אנחנו ל טובה לבר...,0,2552,2
3652,https://www.facebook.com/photo.php?fbid=454613...,2,2555,2
8133,ש כל עם ישראל יהיה גאה ב אתה ו ב ה משפחתך ה נה...,0,2556,2


In [17]:
res_dedup_soreted = res_dedup.sort_values(['type', 'index' ])

In [18]:
res_dedup_soreted_to_split = res_dedup_soreted.reset_index(drop=True).filter(items=['comment','label'])

In [19]:
res_dedup_soreted_to_split.head()

,comment,label
0,ממש כואב ..... אני בכה את היא עם ה משפחה ש לא ...,0
1,איש יקר של אנחנו,0
2,כל ה כבוד ו המון ב הצלחה,0
3,"תל חי , רובי . ב כל ה עצב ה זה היית קרן אור ש...",0
4,נקי כפיים ו בר לבב ב הצלחה ל אתה ו ל אנחנו .,0


In [21]:
import numpy as np
def train_validate_test_split(df, train_percent=.7, validate_percent=.1, seed=None):
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[df.index[:train_end]]
    validate = df.iloc[df.index[train_end:validate_end]]
    test = df.iloc[df.index[validate_end:]]
    return train, validate, test

In [22]:
train, validate, test = train_validate_test_split(res_dedup_soreted_to_split)

In [23]:
train

,comment,label
0,ממש כואב ..... אני בכה את היא עם ה משפחה ש לא ...,0
1,איש יקר של אנחנו,0
2,כל ה כבוד ו המון ב הצלחה,0
3,"תל חי , רובי . ב כל ה עצב ה זה היית קרן אור ש...",0
4,נקי כפיים ו בר לבב ב הצלחה ל אתה ו ל אנחנו .,0
...,...,...
5927,ישששששששששששששששש ב שעה טובה איש מדהים איש אמת...,0
5928,יחי יחי נשיא מדינת ישראל רובי רבלין.אנא מ ימך ...,0
5929,צודקת סליחה התכוונתי ב שם ה יהודים ה שפווים ש ...,1
5930,מתבייש ב אתה ש את היא ה נשיא של אני . חשבתי ש ...,1


In [24]:
print(len(train))
print(len(validate))
print(len(test))

5932
847
1696


In [25]:
assert len(test[test['comment'].str.contains("\t")]) ==  0

In [27]:
train.to_csv(fr'{data_format}\train.tsv', sep='\t', index=False, encoding='utf-8')
validate.to_csv(fr'{data_format}\dev.tsv', sep='\t', index=False, encoding='utf-8')
test.to_csv(fr'{data_format}\test.tsv', sep='\t', index=False, encoding='utf-8')
